# Workspace for Messy Development Work

In [1]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.insert(0, "../../scrapemed")

import re
import warnings
import scrapemed._morehtml as mhtml
from scrapemed.utils import basicBiMap

In [2]:
SAMPLE_TEXT1 = ("Hello, this is a test for removal of regular "
    "MHTML tags such as [MHTML::tag], and also typed MHTML tags such as [MHTML::tag_type::tag_value]. "
    "Anyway, that's about all.")

print(mhtml.remove_mhtml_tags(SAMPLE_TEXT1))

Hello, this is a test for removal of regular MHTML tags such as , and also typed MHTML tags such as . Anyway, that's about all.


In [3]:
#removed from the front: <p id="Par22">
#removed from the end: </p>
SAMPLE_PAR = ('Baseline characteristics for the three studies are shown '
    'in Table&#xA0;<xref rid="Tab1" ref-type="table">1</xref>. A total of 35 subjects were '
    'randomized in Study 1 and 46 in Study 2. In Study 3, 21 subjects were assigned to treatment. '
    'In all three studies, the proportion of males and females was approximately 50%. In Study 3, '
    'the majority (62%) of subjects were White, whereas in Studies 1 and 2, the largest proportion '
    'of subjects were Black (54% and 41%, respectively). One subject in Study 1 discontinued study '
    'drug during the first treatment period due to an inability to swallow study medication, and '
    'two subjects in Study 2 discontinued study drug during the first treatment period due to '
    'difficulties in collecting PK samples; no PK profiling was possible for these two subjects. '
    'All 21 subjects in Study 3 completed treatment and were analyzed for PK metrics and safety.'
    '<table-wrap id="Tab1"><label>Table&#xA0;1</label><caption><p>Baseline characteristics</p>'
    '</caption><table frame="hsides" rules="groups"><thead><tr><th align="left"/><th align="left">'
    'Study 1 [<italic>N&#x2009;</italic>=&#x2009;35]</th><th align="left">Study 2 '
    '[<italic>N&#x2009;</italic>=&#x2009;46]</th><th align="left">Study 3 '
    '[<italic>N&#x2009;</italic>=&#x2009;21]</th></tr></thead><tbody><tr><td align="left">'
    'Sex, male</td><td align="left">18 (51.4)</td><td align="left">25 (54.3)</td><td align="left">'
    '10 (47.6)</td></tr><tr><td align="left">Mean age, (years) (SD)</td><td align="left">35.6 (9.4)'
    '</td><td align="left">39.3 (9.2)</td><td align="left">14.7 (1.8)</td></tr><tr><td align="left">'
    '&#xA0;Age 12&#x2013;14&#xA0;years</td><td align="left">&#x2013;</td><td align="left">&#x2013;'
    '</td><td align="left">10 (47.6)</td></tr><tr><td align="left">&#xA0;Age 15&#x2013;17&#xA0;years'
    '</td><td align="left">&#x2013;</td><td align="left">&#x2013;</td><td align="left">11 (52.4)'
    '</td></tr><tr><td align="left" colspan="4">Race</td></tr><tr><td align="left">&#xA0;White'
    '</td><td align="left">4 (11.4)</td><td align="left">9 (19.6)</td><td align="left">13 (61.9)'
    '</td></tr><tr><td align="left">&#xA0;Black</td><td align="left">19 (54.3)</td><td align="left">'
    '19 (41.3)</td><td align="left">3 (14.3)</td></tr><tr><td align="left">&#xA0;Other</td>'
    '<td align="left">12 (34.3)</td><td align="left">18 (39.1)</td><td align="left">'
    '5 (23.8)</td></tr><tr><td align="left" colspan="4">Ethnicity</td></tr><tr><td align="left">'
    '&#xA0;Hispanic/Latino</td><td align="left">14 (40.0)</td><td align="left">18 (39.1)'
    '</td><td align="left">4 (19.0)</td></tr><tr><td align="left">&#xA0;Non-Hispanic/Latino</td>'
    '<td align="left">21 (60.0)</td><td align="left">28 (60.9)</td><td align="left">17 (81.0)'
    '</td></tr><tr><td align="left">Mean weight, (kg) (SD)</td><td align="left">78.5 (14.1)'
    '</td><td align="left">75.4 (12.5)</td><td align="left">56.2 (9.2)</td></tr><tr><td align="left">'
    'Mean height, (cm) (SD)</td><td align="left">171.2 (11.4)</td><td align="left">169.9 (8.6)</td>'
    '<td align="left">164.3 (7.4)</td></tr><tr><td align="left">Mean BMI, (kg/m<sup>2</sup>) (SD)'
    '</td><td align="left">26.6 (3.0)</td><td align="left">26.0 (3.0)</td><td align="left">&#x2013;'
    '</td></tr></tbody></table><table-wrap-foot><p>Data are expressed as <italic>n'
    '</italic> (%) unless otherwise specified</p><p><italic>BMI</italic> body mass index, '
    '<italic>SD</italic> standard deviation</p></table-wrap-foot></table-wrap>')

In [4]:
ref_map = basicBiMap()

XREF_TAG_NAME = 'xref'
FIGURE_TAG_NAME = 'fig'
TABLE_TAG_NAME = 'table'
TABLEWRAP_TAG_NAME = 'table-wrap'
ALLOWED_TAG_NAMES = [XREF_TAG_NAME, FIGURE_TAG_NAME, TABLE_TAG_NAME, TABLEWRAP_TAG_NAME]

text = SAMPLE_PAR.strip()
text

cleaned_text = ""
#matches an html or xml tag until it either self closes or we reach the closing tag
tag_pattern = r'<([a-zA-Z][\w-]*)\b[^>]*>(.*?)</\1>|<([a-zA-Z][\w-]*)\b[^/>]*/?>'
tag_r = re.compile(tag_pattern, re.DOTALL) #DOTALL used in case of multiline tag spans

while len(text) > 0:
    match = tag_r.search(text)
    if match: #found a tag, append the text prior to the tag and deal w tag

        #EAT NEXT TAG AND MATCH PARTS
        tag_name = match.group(1)
        tag_contents = match.group(2)
        full_tag = match.group()

        if not tag_name in ALLOWED_TAG_NAMES:
            warning_msg = (f"Tag of type {tag_name} found in a text portion of the provided markup language. "
            f"Expected only HTML styling tags, or tags from the following list: {ALLOWED_TAG_NAMES}.")
            if id:
                warning_msg += f" Warning occured in a text section with id: {id}."
            warnings.warn(warning_msg, Warning)

        #ADD CONTENTS PRIOR TO TAG 
        tag_start_index = match.start()
        cleaned_text += text[0:tag_start_index] 
        #also add tag contents if it is an xref.
        if tag_name == 'xref':
            cleaned_text += tag_contents

        #Get reference number for data reference
        ref_num = None
        if full_tag in ref_map.reverse:
            #have we generated a map for this tag before?
            ref_num = ref_map.reverse[full_tag]
        else:
            ref_num = len(ref_map) #new tag, append a new key
            ref_map[ref_num] = full_tag #and fill in the tag value

        data_ref_tag = mhtml.generate_typed_mhtml_tag(tag_type="dataref", string=str(ref_num))
        cleaned_text += f"{data_ref_tag}"

        #eat through the text that was just processed
        text = text[match.end():]

    else: #no more tags to deal with, add the last bits to our output text
        cleaned_text += text
        text = ""
        
print(f"\nCleaned Text----------------------------\n{cleaned_text}\n----------------------------------------")
print(ref_map)



Cleaned Text----------------------------
Baseline characteristics for the three studies are shown in Table&#xA0;1[MHTML::dataref::0]. A total of 35 subjects were randomized in Study 1 and 46 in Study 2. In Study 3, 21 subjects were assigned to treatment. In all three studies, the proportion of males and females was approximately 50%. In Study 3, the majority (62%) of subjects were White, whereas in Studies 1 and 2, the largest proportion of subjects were Black (54% and 41%, respectively). One subject in Study 1 discontinued study drug during the first treatment period due to an inability to swallow study medication, and two subjects in Study 2 discontinued study drug during the first treatment period due to difficulties in collecting PK samples; no PK profiling was possible for these two subjects. All 21 subjects in Study 3 completed treatment and were analyzed for PK metrics and safety.[MHTML::dataref::1]
----------------------------------------
BiMap: {0: '<xref rid="Tab1" ref-type=